In [ ]:
def angle_transfomation(ra, dec, polarization, t_gps, detector_name):
    """
    These code used pycbc and bilby package！
    "Based on the parameters of the input source（ra, dec,polarization） and 
    the given detector((H1,L1,K1...)), output the spherical coordinate angles theta, phi, 
    and psi of the source relative to the given detector."

    parameters:
    ra -- right ascension in radians;  0~2*np.pi
    dec --         declination in radians;   -np.pi~np.pi
    polarization -- polarization of the signal, defaut is zero; 0~2*np.pi
    t_gps -- GPS time
    detector_name -- name of the detector，H1 for Hanford, L1 for Liveston, V1 for Virgo, etc.
    
    returns:
    theta -- angular parameters of the source in spherical coordinates;            0~np.pi
    phi -- angular parameters of the source in spherical coordinates;              0~2*np.pi
    psi -- The polarization angle of the source in the detector's reference frame  0~np.pi
    
    """
    
    from pycbc.detector import Detector
    from bilby.gw.detector import get_empty_interferometer
    from numpy import cos, sin
    detector_bilby = get_empty_interferometer(detector_name)
    detector_pycbc = Detector(detector_name)

    phi = ra - detector_pycbc.gmst_estimate(t_gps)
    theta = np.pi / 2 - dec
    
    cosphi = cos(phi)
    sinphi = sin(phi)
    costheta = cos(theta)
    sintheta = sin(theta)
    cospsi = cos(polarization)  #for convenience，name it "cospsi" rather than "cospolarization"
    sinpsi = sin(polarization)
    m0 = - costheta * cosphi * sinpsi + sinphi * cospsi
    m1 = - costheta * sinphi * sinpsi - cosphi * cospsi
    m2 = sintheta * sinpsi
    m = np.array([m0, m1, m2], dtype=np.float64)
    n0 = - costheta * cosphi * cospsi - sinphi * sinpsi
    n1 = - costheta * sinphi * cospsi + cosphi * sinpsi
    n2 = sintheta * cospsi
    n = np.array([n0, n1, n2], dtype=np.float64)

    omega_view = np.cross(m, n)
    #omega_view source direction vector
    ####################################################################    
    x_detector = detector_bilby.x
    y_detector = detector_bilby.y
    z_detector = np.cross(x_detector,y_detector)
    z_detector = z_detector / np.linalg.norm(z_detector)
    #x,y are unit detector arm vectors，z=x cross y， is the top of detector vector
    ####################################################################    

    theta0 = np.arccos(np.dot(omega_view, z_detector))
    ####################################################################    
    coef_a = omega_view.dot(x_detector)
    coef_b = omega_view.dot(y_detector)
    proj_omega = coef_a * x_detector + coef_b * y_detector  #figure out the projection of omega in xOy plane, to calculate phi
    proj_omega = proj_omega / np.linalg.norm(proj_omega) 
    dotx  = proj_omega.dot(x_detector)
    doty  = proj_omega.dot(y_detector)
    phi0 = np.arccos(dotx)
    if (dotx<0  and doty<0 ) or (dotx>0 and doty<0):
        phi0 = 2 * np.pi - phi0
    ####################################################################    
    rotate_axis = np.cross(omega_view, z_detector)
    R_theta = rotation_matrix(rotate_axis, theta0)
    R_phi = rotation_matrix(z_detector, -phi0)
    m_rotated = np.dot(R_theta, m)
    m_rotated = np.dot(R_phi, m_rotated)
    
    dotx_  = m_rotated.dot(x_detector)
    doty_  = m_rotated.dot(y_detector)
    psi0 = np.arccos(dotx_)
    if (dotx_<0  and doty_<0 ) or (dotx_>0 and doty_<0):
        psi0 = 2 * np.pi - psi0
    return theta0, phi0, psi0
def rotation_matrix(axis, theta):
    """
    calculate the rotating matrix around given axis by theta angle(radians)

    parameters:
    axis -- rotating axis (must be unit vector)
    theta -- rotating angle (radians)

    return:
    3x3 rotation matrix
    """
    axis = np.asarray(axis)
    axis = axis / np.linalg.norm(axis)  # make sure it is unit vector
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    
    return np.array([
        [aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
        [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
        [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]
    ])